In [1]:
from math import sqrt

In [9]:
critics = {
    'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
                  'Superman Returns': 3.5, 'You, Me and Dupree': 2.5, 'The Night Listener': 3.0},
    'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5, 'Just My Luck': 1.5,
                     'Superman Returns': 5.0, 'The Night Listener': 3.0, 'You, Me and Dupree': 3.5},
    'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0, 'Superman Returns': 3.5,
                         'The Night Listener': 4.0},
    'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0, 'The Night Listener': 4.5, 'Superman Returns': 4.0,
                     'You, Me and Dupree': 2.5},
    'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 'Just My Luck': 2.0, 'Superman Returns': 3.0,
                     'The Night Listener': 3.0, 'You, Me and Dupree': 2.0},
    'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 'The Night Listener': 3.0,
                      'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
    'Toby': {'Snakes on a Plane':4.5, 'You, Me and Dupree':1.0, 'Superman Returns':4.0}
}

In [10]:
# simple euclid distance
sqrt(pow(5-4, 2) + pow(4-1, 2))

3.1622776601683795

In [11]:
"""
значение формулы тем больше, чем люди более похожи друг на друга.
Добавить к вычисленному расстоянию 1 (чтобы никогда не делить на 0) и взять обратную величину
функция всегда возвращает значение от 0 до 1, причем 1 получается, когда предпочтения двух людей в точности совпадают.
"""
1/(1+sqrt(pow(5-4,2)+pow(4-1,2)))

0.2402530733520421

In [12]:
def sim_distance(prefs, person1, person2):
    # Получить список предметов, оцененных обоими
    si = {}
    
    for item in prefs[person1]:
        if item in prefs[person2]:
            si[item] = 1
            
    # Если нет ни одной общей оценки, вернуть 0
    if len(si) == 0:
        return 0
        # Сложить квадраты разностей
    sum_of_squares = sum([pow(prefs[person1][item] - prefs[person2][item], 2) for item in prefs[person1] if item in prefs[person2]])
    
    return 1 / (1 + sum_of_squares)

In [13]:
sim_distance(critics, 'Jack Matthews', 'Toby')

0.11764705882352941

In [16]:
"""
Эта функция возвращает значение от –1 до 1. Значение 1 означает, что два человека выставили каждому предмету в точности одинаковые оценки.

Программа для вычисления коэффициента корреляции Пирсона сначала находит фильмы, оцененные обоими критиками,
и вычисляет сумму и сумму квадратов выставленных ими оценок, а также сумму произведений оценок.

На последнем этапе найденные значения используются для вычисления коэффициента корреляции.
"""
def sim_pearson(prefs,p1,p2):
    # Получить список предметов, оцененных обоими
    si = {}
    for item in prefs[p1]:
        if item in prefs[p2]:
            si[item] = 1

    # Найти число схожих элементов
    n = len(si)
    
    # Если нет ни одной общей оценки, вернуть 0
    if n == 0:
        return 0

    # Вычислить сумму всех предпочтений
    sum1 = sum([prefs[p1][it] for it in si])
    sum2 = sum([prefs[p2][it] for it in si])

    # Вычислить сумму квадратов
    sum1Sq = sum([pow(prefs[p1][it], 2) for it in si])
    sum2Sq = sum([pow(prefs[p2][it], 2) for it in si])

    # Вычислить сумму произведений
    pSum = sum([prefs[p1][it] * prefs[p2][it] for it in si])
    
    # Вычислить коэффициент Пирсона
    num = pSum - (sum1 * sum2 / n)
    den = sqrt((sum1Sq - pow(sum1, 2) / n) * (sum2Sq - pow(sum2, 2) / n))

    if den == 0:
        return 0

    r = num / den
            
    return r

In [17]:
sim_pearson(critics, 'Jack Matthews', 'Toby')

0.66284898035987

In [18]:
# Возвращает список наилучших соответствий для человека из словаря prefs.
# Количество результатов в списке и функция подобия – необязательные параметры.
def topMatches(prefs, person, n = 5, similarity = sim_pearson):
    scores=[(similarity(prefs, person, other), other) for other in prefs if other != person]
    
    # Отсортировать список по убыванию оценок
    scores.sort()
    scores.reverse()
    return scores[0:n]

In [20]:
topMatches(critics, 'Toby')

[(0.9912407071619299, 'Lisa Rose'),
 (0.9244734516419049, 'Mick LaSalle'),
 (0.8934051474415647, 'Claudia Puig'),
 (0.66284898035987, 'Jack Matthews'),
 (0.38124642583151164, 'Gene Seymour')]

In [39]:
# Получить рекомендации для заданного человека, пользуясь взвешенным средним
# оценок, данных всеми остальными пользователями
def getRecommendations(prefs, person, similarity = sim_pearson):
    totals = {}
    simSums = {}
    
    for other in prefs:
        # сравнивать меня с собой же не нужно
        if other == person:
            continue
        
        sim = similarity(prefs, person, other)
            
        # игнорировать нулевые и отрицательные оценки
        if sim <= 0:
            continue

        for item in prefs[other]:
    #         оценивать только фильмы, которые я еще не смотрел
            if item not in prefs[person] or prefs[person][item] == 0:
                # Коэффициент подобия * Оценка
                totals.setdefault(item, 0)
                totals[item] += prefs[other][item] * sim
                # Сумма коэффициентов подобия
                simSums.setdefault(item, 0)
                simSums[item] += sim
            
    # Создать нормализованный список
    rankings = [(total / simSums[item], item) for item, total in totals.items()]
    
    # Вернуть отсортированный список
    rankings.sort()
    rankings.reverse()
    return rankings

In [40]:
getRecommendations(critics, 'Toby')

[(3.3477895267131017, 'The Night Listener'),
 (2.8325499182641614, 'Lady in the Water'),
 (2.530980703765565, 'Just My Luck')]